# Google Colab initialization

In [1]:
# !pip install pyyaml==5.1

# import torch
# TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
# CUDA_VERSION = torch.__version__.split("+")[-1]
# print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# # Install detectron2 that matches the above pytorch version
# # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html > /dev/null 
# # If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [2]:
GOOGLE_COLAB = False
# GOOGLE_COLAB = True

In [3]:
if GOOGLE_COLAB == True:
    from google.colab import drive
    drive.mount('/content/drive/')

In [4]:
# if GOOGLE_COLAB == True:
#     !ls '/content/drive/MyDrive/18744/data/Cityscapes/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/Cityscapes/'

#     !unzip -n '/content/drive/MyDrive/18744/data/Cityscapes/gtFine_trainvaltest.zip' -d '/content/data/Cityscapes/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Cityscapes/leftImg8bit_trainvaltest.zip' -d '/content/data/Cityscapes/' > /dev/null
#     !tar -xf '/content/drive/MyDrive/18744/data/Cityscapes/mapped_labels.tar' --skip-old-files --directory '/content/data/Cityscapes/'

#     ##################################################

#     !ls '/content/drive/MyDrive/18744/data/Carla/packaging/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/Carla/'
#     !mkdir '/content/data/Carla/packaging/'

#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package2.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package3.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package4.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package5.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package6.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package7.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package8.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !unzip -n '/content/drive/MyDrive/18744/data/Carla/packaging/package9.zip' -d '/content/data/Carla/packaging/' > /dev/null
#     !tar -xf '/content/drive/MyDrive/18744/data/Carla/packaging/semantic_train.tar' --skip-old-files --directory '/content/data/Carla/packaging/'

#     ##################################################

#     !ls '/content/drive/MyDrive/18744/data/CarlaNight/night_packaging/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/CarlaNight/'
#     !mkdir '/content/data/CarlaNight/night_packaging/'
    
#     !tar -xf '/content/drive/MyDrive/18744/data/CarlaNight/night_packaging/package10.tar' --skip-old-files --directory '/content/data/CarlaNight/night_packaging/'
#     !tar -xf '/content/drive/MyDrive/18744/data/CarlaNight/night_packaging/package11.tar' --skip-old-files --directory '/content/data/CarlaNight/night_packaging/'

#     ##################################################

#     !ls '/content/drive/MyDrive/18744/data/RainAddition/train/'

#     !mkdir '/content/data/'
#     !mkdir '/content/data/RainAddition/'
#     !mkdir '/content/data/RainAddition/train/'

#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/1.tar' --skip-old-files --directory '/content/data/RainAddition/train/'
#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/2.tar' --skip-old-files --directory '/content/data/RainAddition/train/'
#     !tar -xf '/content/drive/MyDrive/18744/data/RainAddition/train/3.tar' --skip-old-files --directory '/content/data/RainAddition/train/'

In [5]:
if GOOGLE_COLAB == True:
    !ls '/content/drive/MyDrive/18744/data/'
    %cd '/content/drive/MyDrive/18744/Robust-Vision-in-Rain/Detectron2Predictor/'
    !ls

# Configs

In [6]:
main_dir = './'
data_dir = '/home/tunx404/Miscellaneous/data/' # Local Jupyter

if GOOGLE_COLAB == True:
    # data_dir = '/content/drive/MyDrive/18744/data/'
    data_dir = '/content/data/'
    
DEBUG = False
DEBUG = True

EVAL_LEVEL_LIST = ['H', 'M', 'S']
# EVAL_LEVEL_LIST = ['H']

# Libraries

In [7]:
##################################################
# Local libraries

from detectron2_predictor import Detectron2Predictor
from detectron2_trainer import Detectron2Trainer
from detectron2_dataset import Detectron2CustomDataset
from utilities import create_file_list, imshow_jupyter
from datasets import get_carla_file_list, get_carla_dicts, get_cityscapes_file_list, get_cityscapes_dicts
from datasets import convert_carla, convert_cityscapes

##################################################
# Libraries

import numpy as np
import torch

import matplotlib.pyplot as plt
from PIL import Image

import os, json, cv2, random

##################################################
# Detectron2

import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog

Number of train images: 0
Number of val images:   0


# Image lists & dicts

## Carla

In [8]:
data_carla_dir = data_dir + 'Carla/packaging/'

# Dir structure:
# <data_carla_dir>
#     packages2
#     packages3
#     ...

data_carla_train_file_list = get_carla_file_list(data_carla_dir, packages=['package2', 'package3', 'package4', 'package5', 'package6', 'package7', 'package9'], levels=['H', 'M', 'S'])
# data_carla_val_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=['H', 'M', 'S'])
data_carla_val_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=EVAL_LEVEL_LIST)

# # Toy example
# data_carla_train_file_list = data_carla_train_file_list[:10]
# data_carla_val_file_list = data_carla_val_file_list[:10]

Number of images: 2990
Number of images: 1173


In [9]:
# data_carla_dir = data_dir + 'RainRemoval/CarlaDerained/'

# # Dir structure:
# # <data_carla_dir>
# #     packages2
# #     packages3
# #     ...

# data_carla_train_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=EVAL_LEVEL_LIST)
# data_carla_val_file_list = get_carla_file_list(data_carla_dir, packages=['package8'], levels=EVAL_LEVEL_LIST)

# # # Toy example
# # data_carla_train_file_list = data_carla_train_file_list[:10]
# # data_carla_val_file_list = data_carla_val_file_list[:10]

In [10]:
print('**************************************************')
print('Carla clear')

def get_carla_clear_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=True, rain=False)

def get_carla_clear_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=True, rain=False)

if DEBUG == True:
    for dict in get_carla_clear_train_dicts()[:4]:
        print(dict)
    
print(len(get_carla_clear_train_dicts())) # 2990
print(len(get_carla_clear_val_dicts()))   # 1173

print('\n**************************************************')
print('Carla rain')

def get_carla_rain_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=False, rain=True)

def get_carla_rain_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=False, rain=True)

if DEBUG == True:
    for dict in get_carla_rain_val_dicts()[:4]:
        print(dict)
    
print(len(get_carla_rain_train_dicts())) # 2990
print(len(get_carla_rain_val_dicts()))   # 1173

print('\n**************************************************')
print('Carla all')

def get_carla_all_train_dicts():
    return get_carla_dicts(data_carla_train_file_list, data_carla_dir, clear=True, rain=True)

def get_carla_all_val_dicts():
    return get_carla_dicts(data_carla_val_file_list, data_carla_dir, clear=True, rain=True)

if DEBUG == True:
    for dict in get_carla_all_val_dicts()[:4]:
        print(dict)
    
print(len(get_carla_all_train_dicts())) # 2990*2 = 5980
print(len(get_carla_all_val_dicts()))   # 1173*2 = 2346

**************************************************
Carla clear
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144040.0070002_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144040.0070002_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144040.0070002_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144051.8223236_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144051.8223236_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144051.8223236_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144063.4320533_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144063.4320533_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144063.4320533_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/p

## Carla night

In [11]:
data_carla_night_dir = data_dir + 'CarlaNight/night_packaging/'

# Dir structure:
# <data_carla_night_dir>
#     packages10
#     packages11
#     ...

data_carla_night_train_file_list = get_carla_file_list(data_carla_night_dir, packages=['package10'], levels=['H', 'M', 'S'])
# data_carla_night_val_file_list = get_carla_file_list(data_carla_night_dir, packages=['package11'], levels=['H', 'M', 'S'])
data_carla_night_val_file_list = get_carla_file_list(data_carla_night_dir, packages=['package11'], levels=EVAL_LEVEL_LIST)

# # Toy example
# data_carla_night_train_file_list = data_carla_night_train_file_list[:10]
# data_carla_night_val_file_list = data_carla_night_val_file_list[:10]

Number of images: 2842
Number of images: 2974


In [12]:
print('**************************************************')
print('Carla night clear')

def get_carla_night_clear_train_dicts():
    return get_carla_dicts(data_carla_night_train_file_list, data_carla_night_dir, clear=True, rain=False)

def get_carla_night_clear_val_dicts():
    return get_carla_dicts(data_carla_night_val_file_list, data_carla_night_dir, clear=True, rain=False)

if DEBUG == True:
    for dict in get_carla_night_clear_val_dicts()[:4]:
        print(dict)
    
print(len(get_carla_night_clear_train_dicts())) # 2842
print(len(get_carla_night_clear_val_dicts()))   # 2974

print('\n**************************************************')
print('Carla night rain')

def get_carla_night_rain_train_dicts():
    return get_carla_dicts(data_carla_night_train_file_list, data_carla_night_dir, clear=False, rain=True)

def get_carla_night_rain_val_dicts():
    return get_carla_dicts(data_carla_night_val_file_list, data_carla_night_dir, clear=False, rain=True)

if DEBUG == True:
    for dict in get_carla_night_rain_val_dicts()[:4]:
        print(dict)
    
print(len(get_carla_night_rain_train_dicts())) # 2842
print(len(get_carla_night_rain_val_dicts()))   # 2974

print('\n**************************************************')
print('Carla night all')

def get_carla_night_all_train_dicts():
    return get_carla_dicts(data_carla_night_train_file_list, data_carla_night_dir, clear=True, rain=True)

def get_carla_night_all_val_dicts():
    return get_carla_dicts(data_carla_night_val_file_list, data_carla_night_dir, clear=True, rain=True)

if DEBUG == True:
    for dict in get_carla_night_all_val_dicts()[:4]:
        print(dict)
    
print(len(get_carla_night_all_train_dicts())) # 2842*2 = 5684
print(len(get_carla_night_all_val_dicts()))   # 2974*2 = 5948

**************************************************
Carla night clear
{'file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004471.747364_clear.png', 'height': 720, 'width': 1280, 'image_id': '1650004471.747364_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004471.747364_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004480.8757513_clear.png', 'height': 720, 'width': 1280, 'image_id': '1650004480.8757513_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004480.8757513_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004489.9787946_clear.png', 'height': 720, 'width': 1280, 'image_id': '1650004489.9787946_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/CarlaNight/night_packaging/package11/1650004489.9787946_s

## Cityscapes

In [13]:
data_dir_cityscapes = data_dir + 'Cityscapes/leftImg8bit/train/'
# data_dir_rain_addition = data_dir + 'RainAddition/train/'
data_dir_rain_addition = data_dir + 'RainRemoval/RainAddition/'
# anno_dir_cityscapes = data_dir + 'Cityscapes/gtFine/train/'
anno_dir_cityscapes = data_dir + 'Cityscapes/mapped_labels/train/'

city_name_list, _ = create_file_list(data_dir_cityscapes)
print(f'Number of cities: {len(city_name_list)}')

data_cityscapes_train_file_list = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[:13])
data_cityscapes_val_file_list   = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list[13:])

# # Toy example
# data_cityscapes_train_file_list = data_cityscapes_train_file_list[:10]
# data_cityscapes_val_file_list = data_cityscapes_val_file_list[:10]

Number of cities: 18
Number of images: 2276
Number of images: 699


In [14]:
# data_dir_cityscapes = data_dir + 'RainRemoval/RainAddition/'
# # data_dir_rain_addition = data_dir + 'RainAddition/train/'
# data_dir_rain_addition = data_dir + 'RainRemoval/RainAddition/'
# # anno_dir_cityscapes = data_dir + 'Cityscapes/gtFine/train/'
# anno_dir_cityscapes = data_dir + 'Cityscapes/mapped_labels/train/'

# city_name_list, _ = create_file_list(data_dir_cityscapes)
# print(f'Number of cities: {len(city_name_list)}')

# data_cityscapes_train_file_list = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list)
# data_cityscapes_val_file_list   = get_cityscapes_file_list(data_dir_cityscapes, cities=city_name_list)

# # # Toy example
# # data_cityscapes_train_file_list = data_cityscapes_train_file_list[:10]
# # data_cityscapes_val_file_list = data_cityscapes_val_file_list[:10

# def get_cityscapes_dicts(file_list, data_dir_main, data_dir_rain, anno_dir, clear=True, rain=True, levels=[]):
#     dicts = []
    
#     # for file in file_list:
#     for index, file in enumerate(file_list):
#         file_name, city = file

#         image_clear_path = os.path.join(data_dir_main, city, file_name)
        
#         file_name_split = file_name.split('_')
        
#         level = file_name_split[3][0]
        
#         # anno_name = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2] + '_gtFine_labelIds.png' # Default
#         image_id = file_name_split[0] + '_' + file_name_split[1] + '_' + file_name_split[2]
#         anno_name = image_id + '_train.png' # Mapped
#         image_semantic_path = os.path.join(anno_dir, city, anno_name)
        
#         if level in levels:
#             if clear == True:
#                 record = {}
#                 record['file_name'] = image_clear_path
#                 record['height'] = 1024 # shape[0]
#                 record['width'] = 2048 # shape[1]
#                 record['image_id'] = image_id + '_clear'
#                 record['sem_seg_file_name'] = image_semantic_path
#                 dicts.append(record)

#             if rain == True:
#                 image_rain_name = image_id + '_' + level + '.png'
#                 image_rain_path = os.path.join(data_dir_rain, city, image_rain_name)

#                 record = {}
#                 record['file_name'] = image_rain_path
#                 record['height'] = 1024
#                 record['width'] = 2048
#                 record['image_id'] = image_id + '_rain'
#                 record['sem_seg_file_name'] = image_semantic_path
#                 dicts.append(record)
#     return dicts

In [15]:
print('**************************************************')
print('Cityscapes clear')

def get_cityscapes_clear_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=False, levels=['H', 'M', 'S'])

def get_cityscapes_clear_val_dicts():
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=False, levels=['H', 'M', 'S'])

if DEBUG == True:
    for dict in get_cityscapes_clear_val_dicts()[:4]:
        print(dict)
    
print(len(get_cityscapes_clear_train_dicts())) # 2276
print(len(get_cityscapes_clear_val_dicts()))   # 699

print('\n**************************************************')
print('Cityscapes rain')

def get_cityscapes_rain_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=False, rain=True, levels=['H', 'M', 'S'])

def get_cityscapes_rain_val_dicts():
    # return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=False, rain=True, levels=['H', 'M', 'S'])
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=False, rain=True, levels=EVAL_LEVEL_LIST)

if DEBUG == True:
    for dict in get_cityscapes_rain_val_dicts()[:4]:
        print(dict)
    
print(len(get_cityscapes_rain_train_dicts())) # 2276
print(len(get_cityscapes_rain_val_dicts()))   # 699

print('\n**************************************************')
print('Cityscapes all')

def get_cityscapes_all_train_dicts():
    return get_cityscapes_dicts(data_cityscapes_train_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=True, levels=['H', 'M', 'S'])

def get_cityscapes_all_val_dicts():
    return get_cityscapes_dicts(data_cityscapes_val_file_list, data_dir_cityscapes, data_dir_rain_addition, anno_dir_cityscapes, clear=True, rain=True, levels=['H', 'M', 'S'])

if DEBUG == True:
    for dict in get_cityscapes_all_val_dicts()[:4]:
        print(dict)

print(len(get_cityscapes_all_train_dicts())) # 2276*2 = 4552
print(len(get_cityscapes_all_val_dicts()))   # 699*2  = 1398

**************************************************
Cityscapes clear
{'file_name': '/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit/train/stuttgart/stuttgart_000000_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'stuttgart_000000_000019_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/stuttgart/stuttgart_000000_000019_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit/train/stuttgart/stuttgart_000001_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'stuttgart_000001_000019_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/stuttgart/stuttgart_000001_000019_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Cityscapes/leftImg8bit/train/stuttgart/stuttgart_000002_000019_leftImg8bit.png', 'height': 1024, 'width': 2048, 'image_id': 'stuttgart_000002_000019_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/dat

## Combined

In [16]:
print('**************************************************')
print('Combined clear')

def get_combined_clear_train_dicts():
    return get_carla_clear_train_dicts() + get_cityscapes_clear_train_dicts()

def get_combined_clear_val_dicts():
    return get_carla_clear_val_dicts() + get_cityscapes_clear_val_dicts()

if DEBUG == True:
    for dict in get_combined_clear_train_dicts()[:4]:
        print(dict)
    for dict in get_combined_clear_train_dicts()[-4:]:
        print(dict)
    
print(len(get_combined_clear_train_dicts())) # 2990 + 2276 = 5266
print(len(get_combined_clear_val_dicts()))   # 1173 + 699  = 1872

print('\n**************************************************')
print('Combined rain')

def get_combined_rain_train_dicts():
    return get_carla_rain_train_dicts() + get_cityscapes_rain_train_dicts()

def get_combined_rain_val_dicts():
    return get_carla_rain_val_dicts() + get_cityscapes_rain_val_dicts()

if DEBUG == True:
    for dict in get_combined_rain_train_dicts()[:4]:
        print(dict)
    for dict in get_combined_rain_train_dicts()[-4:]:
        print(dict)
    
print(len(get_combined_rain_train_dicts())) # 2990 + 2276 = 5266
print(len(get_combined_rain_val_dicts()))   # 1173 + 699  = 1872

print('\n**************************************************')
print('Combined all')

def get_combined_all_train_dicts():
    return get_carla_all_train_dicts() + get_cityscapes_all_train_dicts()

def get_combined_all_val_dicts():
    return get_carla_all_val_dicts()[:1000] + get_cityscapes_all_val_dicts()[:1000]

if DEBUG == True:
    for dict in get_combined_all_train_dicts()[:4]:
        print(dict)
    for dict in get_combined_all_train_dicts()[-4:]:
        print(dict)

print(len(get_combined_all_train_dicts())) # 5266*2 = 10532
print(len(get_combined_all_val_dicts()))   # 1872*2 = 3744

print('\n**************************************************')
print('Combined all night')

def get_combined_all_night_train_dicts():
    return get_combined_all_train_dicts() + get_carla_night_all_train_dicts()

def get_combined_all_night_val_dicts():
    return get_combined_all_val_dicts() + get_carla_night_all_val_dicts()[:1000]

if DEBUG == True:
    for dict in get_combined_all_night_train_dicts()[:4]:
        print(dict)
    for dict in get_combined_all_night_val_dicts()[-4:]:
        print(dict)

print(len(get_combined_all_night_train_dicts())) # 10532 + 5684 = 16216
print(len(get_combined_all_night_val_dicts()))   # 3744  + 5948 = 9692

**************************************************
Combined clear
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144040.0070002_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144040.0070002_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144040.0070002_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144051.8223236_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144051.8223236_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144051.8223236_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144063.4320533_clear.png', 'height': 720, 'width': 1280, 'image_id': '1647144063.4320533_clear', 'sem_seg_file_name': '/home/tunx404/Miscellaneous/data/Carla/packaging/package2/1647144063.4320533_semantic_train.png'}
{'file_name': '/home/tunx404/Miscellaneous/data/Carl

# Convert labels to train IDs
Execute only once

In [17]:
# %cd '/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/'
# for city_name in city_name_list:
#     !mkdir $city_name

In [18]:
# convert_carla(data_carla_train_file_list, data_carla_dir)
# convert_carla(data_carla_val_file_list, data_carla_dir)

# convert_carla(data_carla_night_train_file_list, data_carla_night_dir)
# convert_carla(data_carla_night_val_file_list, data_carla_night_dir)

# convert_cityscapes(data_cityscapes_train_file_list, anno_dir_cityscapes, output_dir='/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/')
# convert_cityscapes(data_cityscapes_val_file_list, anno_dir_cityscapes, output_dir='/home/tunx404/Miscellaneous/data/Cityscapes/mapped_labels/train/')

# Datasets

In [45]:
DatasetCatalog.clear()

print('**************************************************')

print('Carla clear dataset')
carla_clear_dataset = Detectron2CustomDataset('carla_clear_train', 'carla_clear_val', get_carla_clear_train_dicts, get_carla_clear_val_dicts)
# carla_clear_dataset.visualize_dataset(num_samples=4, size=(20, 10), show_original=True)
            
print('\nCarla rain dataset')
carla_rain_dataset = Detectron2CustomDataset('carla_rain_train', 'carla_rain_val', get_carla_rain_train_dicts, get_carla_rain_val_dicts)
# carla_rain_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)

# print('\nCarla derained dataset')
# carla_derained_dataset = Detectron2CustomDataset('carla_derained_train', 'carla_derained_val', get_carla_rain_train_dicts, get_carla_derained_val_dicts)
# # carla_rain_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)

##################################################

print('\n**************************************************')

print('Carla night clear dataset')
carla_night_clear_dataset = Detectron2CustomDataset('carla_night_clear_train', 'carla_night_clear_val', get_carla_night_clear_train_dicts, get_carla_night_clear_val_dicts)
# carla_night_clear_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)
            
print('\nCarla night rain dataset')
carla_night_rain_dataset = Detectron2CustomDataset('carla_night_rain_train', 'carla_night_rain_val', get_carla_night_rain_train_dicts, get_carla_night_rain_val_dicts)
# carla_night_rain_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)

##################################################

print('\n**************************************************')

print('Cityscapes clear dataset')
cityscapes_clear_dataset = Detectron2CustomDataset('cityscapes_clear_train', 'cityscapes_clear_val', get_cityscapes_clear_train_dicts, get_cityscapes_clear_val_dicts)
# cityscapes_clear_dataset.visualize_dataset(num_samples=10, size=(20, 10), show_original=True)

print('\nCityscapes rain dataset')
cityscapes_rain_dataset = Detectron2CustomDataset('cityscapes_rain_train', 'cityscapes_rain_val', get_cityscapes_rain_train_dicts, get_cityscapes_rain_val_dicts)
# cityscapes_rain_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)

##################################################

print('\n**************************************************')
            
print('Combined clear dataset')
combined_clear_dataset = Detectron2CustomDataset('combined_clear_train', 'combined_clear_val', get_combined_clear_train_dicts, get_combined_clear_val_dicts)
# combined_clear_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)
            
print('\nCombined rain dataset')
combined_rain_dataset = Detectron2CustomDataset('combined_rain_train', 'combined_rain_val', get_combined_rain_train_dicts, get_combined_rain_val_dicts)
# combined_rain_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)
            
print('\nCombined all dataset')
combined_all_dataset = Detectron2CustomDataset('combined_all_train', 'combined_all_val', get_combined_all_train_dicts, get_combined_all_val_dicts)
# combined_all_dataset.visualize_dataset(num_samples=1, size=(20, 10), show_original=True)

##################################################

print('\n**************************************************')
            
print('Combined all night dataset')
combined_all_night_dataset = Detectron2CustomDataset('combined_all_night_train', 'combined_all_night_val', get_combined_all_night_train_dicts, get_combined_all_night_val_dicts)
# combined_all_dataset.visualize_dataset(num_samples=20, size=(20, 10), show_original=True)

**************************************************
Carla clear dataset
Number of train images: 2990
Number of val images:   1173

Carla rain dataset
Number of train images: 2990
Number of val images:   1173

**************************************************
Carla night clear dataset
Number of train images: 2842
Number of val images:   2974

Carla night rain dataset
Number of train images: 2842
Number of val images:   2974

**************************************************
Cityscapes clear dataset
Number of train images: 2276
Number of val images:   699

Cityscapes rain dataset
Number of train images: 2276
Number of val images:   699

**************************************************
Combined clear dataset
Number of train images: 5266
Number of val images:   1872

Combined rain dataset
Number of train images: 5266
Number of val images:   1872

Combined all dataset
Number of train images: 10532
Number of val images:   2000

**************************************************
Combined a

# Train

In [20]:
# trainer_all = Detectron2Trainer('combined_all_night_train', 'combined_all_night_val', output_folder='./output_all_night_40k')
# trainer_all.load()
# trainer_all.train()

In [21]:
# trainer_clear = Detectron2Trainer('cityscapes_clear_train', 'cityscapes_clear_val', output_folder='./output_cityscapes_clear_20k')
# trainer_clear.load()
# trainer_clear.train()

In [22]:
# %load_ext tensorboard
# %tensorboard --logdir output_all_40k

In [23]:
# %load_ext tensorboard
# %tensorboard --logdir output_clear_20k

# Evaluate

## 3. Model trained on all with carla night

In [24]:
# trainer_all = Detectron2Trainer('combined_all_night_train', 'carla_rain_val', output_folder='./output_combined_night_all_40k')
# trainer_all.test()

In [25]:
# trainer_all = Detectron2Trainer('combined_all_night_train', 'carla_night_rain_val', output_folder='./output_combined_night_all_40k')
# trainer_all.test()

In [26]:
# trainer_all = Detectron2Trainer('combined_all_night_train', 'cityscapes_rain_val', output_folder='./output_combined_night_all_40k')
# trainer_all.test()

In [27]:
# trainer_all = Detectron2Trainer('combined_all_night_train', 'cityscapes_clear_val', output_folder='./output_combined_night_all_40k')
# trainer_all.test()

## 2. Model trained on all

In [28]:
# trainer_all = Detectron2Trainer('combined_all_train', 'combined_all_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [29]:
# trainer_all = Detectron2Trainer('combined_all_train', 'combined_clear_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [30]:
# trainer_all = Detectron2Trainer('combined_all_train', 'combined_rain_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [31]:
# trainer_all = Detectron2Trainer('combined_all_train', 'carla_rain_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [32]:
# trainer_all = Detectron2Trainer('combined_all_train', 'cityscapes_rain_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [33]:
# trainer_all = Detectron2Trainer('combined_all_train', 'carla_clear_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

In [34]:
# trainer_all = Detectron2Trainer('combined_all_train', 'carla_night_rain_val', output_folder='./output_combined_all_40k')
# trainer_all.test()

## 1. Model trained on clear

In [35]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'combined_all_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [36]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'combined_clear_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [37]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'combined_rain_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [38]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'carla_rain_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [39]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'cityscapes_rain_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [40]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'carla_clear_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()

In [41]:
# trainer_clear = Detectron2Trainer('combined_clear_train', 'carla_night_rain_val', output_folder='./output_combined_clear_20k')
# trainer_clear.test()